## Library

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import matplotlib.pyplot as plt
import tqdm
import wandb
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
import optuna
import regex
import json

## WanDB

In [ ]:
run = str(input('run 이름을 입력하세요 :'))
# selected_model = str(input('model 명을 입력하세요 (xgb/rf) :'))
opt = bool(input('Optuna 사용 여부를 입력하세요 (뭐라도 입력 시 사용) :'))

wandb.init(
    settings=wandb.Settings(start_method='thread'),
    dir=None,  # 로컬에 로그 저장하지 않음
    entity='remember-us', # team name,
    project='active', # project name
    name=run, # run name
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: davinkeem (remember-us). Use `wandb login --relogin` to force relogin


## Data Load

In [ ]:
data_path: str = '/data/ephemeral/home/book/data/'
users = pd.read_csv(data_path + 'users.csv')
books = pd.read_csv(data_path + 'books.csv')
train = pd.read_csv(data_path + 'train_ratings.csv')
test = pd.read_csv(data_path + 'test_ratings.csv')
sub = pd.read_csv(data_path + 'sample_submission.csv')

## Data Preprocessing

In [4]:
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    return x[2:-2].split(', ')

In [5]:
def split_location(x: str) -> list:
    '''
    Parameters
    ----------
    x : str
        location 데이터

    Returns
    -------
    res : list
        location 데이터를 나눈 뒤, 정제한 결과를 반환합니다.
        순서는 country, state, city, ... 입니다.
    '''
    res = x.split(',')
    res = [i.strip().lower() for i in res]
    res = [regex.sub(r'[^a-zA-Z/ ]', '', i) for i in res]  # remove special characters
    res = [i if i not in ['n/a', ''] else np.nan for i in res]  # change 'n/a' into NaN
    res.reverse()  # reverse the list to get country, state, city, ... order

    for i in range(len(res)-1, 0, -1):
        if (res[i] in res[:i]) and (not pd.isna(res[i])):  # remove duplicated values if not NaN
            res.pop(i)

    return res

In [ ]:
def text_preprocessing(summary_text: str) -> str:
    '''
    주어진 텍스트 요약을 전처리합니다.

    1. 특수 문자 제거
    2. 알파벳과 숫자, 공백을 제외한 모든 문자 제거
    3. 여러 개의 공백을 하나의 공백으로
    4. 문자열의 앞뒤 공백 제거
    5. 모든 문자를 소문자로 변환

    Args:
        summary_text (str): 전처리할 텍스트 문자열

    Returns:
        str: 전처리된 텍스트 문자열. 입력이 NaN인 경우 'unknown' 반환.
    '''
    if pd.isna(summary_text):
        return 'unknown'  # NaN일 경우 'unknown' 반환
    
    summary_text = regex.sub('[.,\'\'''\'!?]', '', summary_text)  # 특수 문자 제거
    summary_text = regex.sub('[^0-9a-zA-Z\s]', '', summary_text)  # 알파벳과 숫자, 공백 제외한 문자 제거
    summary_text = regex.sub('\s+', ' ', summary_text)  # 여러 개의 공백을 하나의 공백으로
    summary_text = summary_text.lower()  # 소문자로 변환
    summary_text = summary_text.strip()  # 앞뒤 공백 제거
    return summary_text

In [ ]:
def categorize_publication(x: int, a: int) -> int:
    '''
    주어진 연도를 특정 기준에 따라 카테고리화하는 함수입니다.

    Parameters
    ----------
    x : int
        책의 발행 연도.
    a : int
        연도를 그룹화할 때 사용할 기준값 (예: 5년 단위로 그룹화).

    Returns
    -------
    int
        카테고리화된 연도를 반환합니다. 
        - 1970년 이하의 연도는 1970으로 반환합니다.
        - 2000년 초과의 연도는 2006으로 반환합니다.
        - 나머지 연도는 a 값에 맞게 그룹화하여 반환합니다.

    Example
    -------
    books['years'] = books['year_of_publication'].apply(lambda x: categorize_publication(x, 5))
    print(books['years'].value_counts())
    '''
    if x <= 1970:
        return 1970
    elif x > 2000:
        return 2006
    else:
        return x // a * a

In [ ]:
def extract_language_from_isbn(isbn):
    '''
    ISBN 정보를 사용하여 언어 코드를 추출하는 함수입니다.

    Parameters
    ----------
    isbn : str
        책의 ISBN 번호.

    Returns
    -------
    str
        ISBN에서 추출한 언어 코드. ISBN이 비어있거나 형식에 맞지 않을 경우 최빈값 'en'을 반환합니다.
        - isbn_language_map 참고
        - 기타 언어 코드: isbn_language_map에 정의된 국가 코드를 기반으로 반환
    '''
    isbn_language_map = {
        '0': 'en', '1': 'en', '2': 'fr', '3': 'de', '4': 'ja',
        '5': 'ru', '7': 'zh-CN', '82': 'no', '84': 'es', '87': 'da',
        '88': 'it', '89': 'ko', '94': 'nl', '600': 'fa', '602': 'ms',
        '606': 'ro', '604': 'vi', '618': 'el', '967': 'ms', '974': 'th',
        '989': 'pt'
    }
    if not isbn or not isbn.isdigit():
        return 'en'  # 기본값 영어권
    for prefix, language in isbn_language_map.items():
        if isbn.startswith(prefix):
            return language
    return 'en'  # 기본값 영어권

In [ ]:
def replace_language_using_isbn(books):
    '''
    ISBN 정보를 활용하여 language 결측치를 대체하는 함수입니다.

    Parameters
    ----------
    books : pd.DataFrame
        책 정보가 담긴 DataFrame. 반드시 'isbn' 및 'language' 열을 포함해야 합니다.

    Returns
    -------
    pd.DataFrame
        language 결측치가 ISBN 정보를 사용해 대체된 DataFrame. ISBN에서 언어를 추출할 수 없는 경우
        기본값 'en'으로 대체됩니다.

    Example
    -------
    books = replace_language_using_isbn(books)
    '''
    books['extracted_language'] = books['isbn'].apply(extract_language_from_isbn)
    books['language'] = books.apply(
        lambda row: row['extracted_language'] if pd.isna(row['language']) else row['language'],
        axis=1
    )
    books.drop(columns=['extracted_language'], inplace=True)
    return books

In [ ]:
def categorize_age(x: int, a: int) -> int:
    '''
    주어진 나이를 특정 기준에 따라 카테고리화하는 함수입니다.

    Parameters
    ----------
    x : int
        유저의 나이.
    a : int
        나이를 그룹화할 때 사용할 기준값 (예: 10년 단위로 그룹화).

    Returns
    -------
    int
        카테고리화된 나이를 반환합니다. 
        - 20년 미만의 나이는 10으로 반환합니다.
        - 60년 이상의 나이는 60으로 반환합니다.
        - 나머지 나이는 a 값에 맞게 그룹화하여 반환합니다.
    '''
    if x < 20:
        return 10
    elif x >= 60:
        return 60
    else:
        return x // a * a

In [11]:
users_ = users.copy()
books_ = books.copy()

In [ ]:
# books 데이터 전처리
books_['book_title'] = books_['book_title'].apply(text_preprocessing)
books_['book_author'] = books_['book_author'].apply(text_preprocessing)
books_['publisher'] = books_['publisher'].apply(text_preprocessing)
books_['publication_range'] = books_['year_of_publication'].apply(lambda x: categorize_publication(x, 5))
books_ = replace_language_using_isbn(books_)
books_['category'] = books_['category'].apply(lambda x: str2list(x)[0] if not pd.isna(x) else np.nan)
books_['category'] = books_['category'].apply(text_preprocessing)
high_categories = ['fiction', 'biography', 'history', 'religion', 'nonfiction', 'social', 'science', 'humor', 'body', 
                'business', 'economics', 'cook', 'health', 'fitness', 'famil', 'relationship', 
                'computer', 'travel', 'selfhelp', 'psychology', 'poetry', 'art', 'critic', 'nature', 'philosophy', 
                'reference','drama', 'sports', 'politic', 'comic', 'novel', 'craft', 'language', 'education', 'crime', 'music', 'pet', 
                'child', 'collection', 'mystery', 'garden', 'medical', 'author', 'house','technology', 'engineering', 'animal', 'photography',
                'adventure', 'game', 'science fiction', 'architecture', 'law', 'fantasy', 'antique', 'friend', 'brother', 'sister', 'cat',
                'math', 'christ', 'bible', 'fairy', 'horror', 'design', 'adolescence', 'actor', 'dog', 'transportation', 'murder', 'adultery', 'short', 'bear'
                ]
# high_category 열을 초기화
books_['high_category'] = None
# 각 카테고리에 대해 반복하며 매핑
for high_category in high_categories:
    # category 열에서 high_category가 포함된 행을 찾고, 해당 행의 high_category 열을 업데이트
    books_.loc[books_['category'].str.contains(high_category, case=False, na=False), 'high_category'] = high_category
books_['high_category'] = books_['high_category'].fillna('others') # 결측치를 'others'로 대체

# users 데이터 전처리
users_['age'] = users_['age'].fillna(users_['age'].mean())
users_['age_range'] = users_['age'].apply(lambda x: categorize_age(x, 10))

users_['location_list'] = users_['location'].apply(lambda x: split_location(x)) 
users_['location_country'] = users_['location_list'].apply(lambda x: x[0])
users_['location_state'] = users_['location_list'].apply(lambda x: x[1] if len(x) > 1 else np.nan)
users_['location_city'] = users_['location_list'].apply(lambda x: x[2] if len(x) > 2 else np.nan)
for idx, row in users_.iterrows():
    if (not pd.isna(row['location_state'])) and pd.isna(row['location_country']):
        fill_country = users_[users_['location_state'] == row['location_state']]['location_country'].mode()
        fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
        users_.loc[idx, 'location_country'] = fill_country
    elif (not pd.isna(row['location_city'])) and pd.isna(row['location_state']):
        if not pd.isna(row['location_country']):
            fill_state = users_[(users_['location_country'] == row['location_country']) 
                                & (users_['location_city'] == row['location_city'])]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            users_.loc[idx, 'location_state'] = fill_state
        else:
            fill_state = users_[users_['location_city'] == row['location_city']]['location_state'].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            fill_country = users_[users_['location_city'] == row['location_city']]['location_country'].mode()
            fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
            users_.loc[idx, 'location_country'] = fill_country
            users_.loc[idx, 'location_state'] = fill_state

In [13]:
# location_country 열의 최빈값을 계산
most_frequent_country = users_['location_country'].mode()[0]
# NaN 값을 최빈값으로 대체
users_['location_country'] = users_['location_country'].fillna(most_frequent_country)

In [14]:
users_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           68092 non-null  int64  
 1   location          68092 non-null  object 
 2   age               68092 non-null  float64
 3   age_range         68092 non-null  float64
 4   location_list     68092 non-null  object 
 5   location_country  68092 non-null  object 
 6   location_state    67368 non-null  object 
 7   location_city     65305 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.2+ MB


In [15]:
books_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             149570 non-null  object 
 7   category             149570 non-null  object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
 10  publication_range    149570 non-null  float64
 11  high_category        149570 non-null  object 
dtypes: float64(2), object(10)
memory usage: 13.7+ MB


In [18]:
users_final = users_[['user_id', 'age_range', 'location_country']]
books_final = books_[['isbn', 'book_title', 'book_author', 'publisher', 'language', 'high_category', 'publication_range']]

## Model

In [19]:
# 데이터 병합
train = train.merge(users_final, on='user_id').merge(books_final, on='isbn')
test = test.merge(users_final, on='user_id').merge(books_final, on='isbn')

In [20]:
train.columns

Index(['user_id', 'isbn', 'rating', 'age_range', 'location_country',
       'book_title', 'book_author', 'publisher', 'language', 'high_category',
       'publication_range'],
      dtype='object')

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            306795 non-null  int64  
 1   isbn               306795 non-null  object 
 2   rating             306795 non-null  int64  
 3   age_range          306795 non-null  float64
 4   location_country   306795 non-null  object 
 5   book_title         306795 non-null  object 
 6   book_author        306795 non-null  object 
 7   publisher          306795 non-null  object 
 8   language           306795 non-null  object 
 9   high_category      306795 non-null  object 
 10  publication_range  306795 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 25.7+ MB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            76699 non-null  int64  
 1   isbn               76699 non-null  object 
 2   rating             76699 non-null  int64  
 3   age_range          76699 non-null  float64
 4   location_country   76699 non-null  object 
 5   book_title         76699 non-null  object 
 6   book_author        76699 non-null  object 
 7   publisher          76699 non-null  object 
 8   language           76699 non-null  object 
 9   high_category      76699 non-null  object 
 10  publication_range  76699 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 6.4+ MB


In [ ]:
user_id_counts = train['user_id'].value_counts()
train['user_review_counts'] = train['user_id'].map(user_id_counts)
test['user_review_counts'] = test['user_id'].map(user_id_counts)
test['user_review_counts'] = test['user_review_counts'].fillna(0)

In [24]:
book_isbn_counts = train['isbn'].value_counts()
train['book_review_counts'] = train['isbn'].map(book_isbn_counts)
test['book_review_counts'] = test['isbn'].map(book_isbn_counts)
test['book_review_counts'] = test['book_review_counts'].fillna(0)

In [25]:
cat_col = ['isbn', 'book_title', 'book_author', 'publisher', 'language', 'high_category', 'publication_range', 'user_id', 'age_range', 'location_country']
num_col = ['rating', 'user_review_counts', 'book_review_counts']

for df in [train, test] :
    for cat in cat_col :
        df[cat] = df[cat].astype('str')
    for num in num_col :
        df[num] = df[num].astype('int')


In [28]:
for col in cat_col:
    combined_values = pd.concat([train[col], test[col]]).unique()
    train[col] = pd.Categorical(train[col], categories=combined_values).codes
    test[col] = pd.Categorical(test[col], categories=combined_values).codes

In [32]:
# METRIC 함수
def calculate_metrics(y_true, y_pred):
    metrics = {
        'RMSE' : root_mean_squared_error(y_true, y_pred),
        'MSE' : mean_squared_error(y_true, y_pred),
        'MAE' : mean_absolute_error(y_true, y_pred)
    }
    return metrics

In [ ]:
def skf_train(X_data, y_data, params):
    skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    valid_rmse = []
    valid_mse = []
    valid_mae = []
    pred_df = pd.DataFrame()

    for fold, (train_idx, valid_idx) in tqdm.tqdm(enumerate(skf.split(X_data, y_data)), total = skf.n_splits) : 
        
        # Train Set과 Valid Set 분할    
        X_train, y_train = X_data.iloc[train_idx], y_data.iloc[train_idx]
        X_valid, y_valid = X_data.iloc[valid_idx], y_data.iloc[valid_idx]
        
        train_data = Pool(data = X_train, label = y_train, cat_features = cat_col)
        valid_data = Pool(data = X_valid, label = y_valid, cat_features = cat_col)
        
        cat_model = CatBoostRegressor(**params, iterations = 1000, 
                                    loss_function = 'RMSE', eval_metric = 'RMSE', 
                                    use_best_model = True, random_state = 42,
                                    cat_features = [i for i in range(0, 10)])
        cat_model.fit(train_data, eval_set = [train_data, valid_data], use_best_model = True, verbose = 100, early_stopping_rounds = 100)
        
        # 모델 RMSE
        valid_metrics = calculate_metrics(y_valid, cat_model.predict(X_valid))
        print(f'Fold {fold + 1} Valid RMSE: {valid_metrics['RMSE']}')
        print(f'Fold {fold + 1} Valid MSE:  {valid_metrics['MSE']}')
        print(f'Fold {fold + 1} Valid MAE:  {valid_metrics['MAE']}')
        valid_rmse.append(valid_metrics['RMSE'])
        valid_mse.append(valid_metrics['MSE'])
        valid_mae.append(valid_metrics['MAE'])

        wandb.log({
            'Valid RMSE': valid_metrics['RMSE'],
            'Valid MSE': valid_metrics['MSE'],
            'Valid MAE': valid_metrics['MAE']
        })
        
        # Predict
        pred = cat_model.predict(test.drop(['rating'], axis = 1))
        pred_df[f'pred_{fold}'] = pred
        
    print(f'RMSE 평균 : {np.array(valid_rmse).mean():.4f} \n')

    params = json.dumps(params)
    wandb.log({
        'Valid RMSE': np.array(valid_rmse).mean(),
        'Valid MSE': np.array(valid_mse).mean(),
        'Valid MAE': np.array(valid_mae).mean(),
        'param': params,
        'features': list(X_data.columns)
    })
    wandb.finish()

    return pred_df

In [ ]:
def optuna_train(X_data, y_data):
    def train(X_data, y_data, params):
        
        # Train Set과 Valid Set 분할    
        X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)
        
        train_data = Pool(data = X_train, label = y_train, cat_features = cat_col)
        valid_data = Pool(data = X_valid, label = y_valid, cat_features = cat_col)
        
        cat_model = CatBoostRegressor(**params, iterations = 500, 
                                    loss_function = 'RMSE', eval_metric = 'RMSE', 
                                    use_best_model = True, random_state = 42,
                                    cat_features = [i for i in range(0, 10)])
        cat_model.fit(train_data, eval_set = [train_data, valid_data], use_best_model = True,
                    verbose = 500, early_stopping_rounds = 100)
        
        valid_metrics = calculate_metrics(y_valid, cat_model.predict(X_valid))

        return valid_metrics['RMSE']

    def objective(trial):
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'depth': trial.suggest_int('depth', 3, 10),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
            'boosting_type': 'Plain',
            'bootstrap_type': 'MVS',
            'devices': 'cuda',
        }
        return train(X_data, y_data, params=params)
        
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='minimize', sampler=sampler)
    study.optimize(objective, n_trials=50)
    return study.best_params

In [ ]:
X_data, y_data = train.drop(columns = 'rating'), train['rating']

if opt:
    best_params = optuna_train(X_data, y_data)
else:
    best_params = {
        'learning_rate': 0.5,
        'depth': 7,
        'colsample_bylevel': 0.5,
        'boosting_type': 'Plain',
        'bootstrap_type': 'MVS',
        'devices': 'cuda',
    }
pred_df = skf_train(X_data, y_data, params=best_params)

  0%|          | 0/10 [00:00<?, ?it/s]

0:	learn: 2.3089768	test: 2.2250116	test1: 2.2853245	best: 2.2853245 (0)	total: 366ms	remaining: 30m 27s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2.131574843
bestIteration = 121

Shrink model to first 122 iterations.
Fold 1 Valid RMSE: 2.131574842982223
Fold 1 Valid MSE:  4.54361131123469
Fold 1 Valid MAE:  1.6060710603646966


 10%|█         | 1/10 [01:00<09:03, 60.44s/it]

0:	learn: 2.3086276	test: 2.2304251	test1: 2.2819052	best: 2.2819052 (0)	total: 262ms	remaining: 21m 50s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2.121281341
bestIteration = 125

Shrink model to first 126 iterations.
Fold 2 Valid RMSE: 2.1212813406764597
Fold 2 Valid MSE:  4.499834526302117
Fold 2 Valid MAE:  1.5987580005170183


 20%|██        | 2/10 [02:02<08:10, 61.34s/it]

...

 90%|█████████ | 9/10 [09:50<01:03, 63.06s/it]

0:	learn: 2.3037831	test: 2.1710382	test1: 2.2778358	best: 2.2778358 (0)	total: 273ms	remaining: 22m 45s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2.132075392
bestIteration = 149

Shrink model to first 150 iterations.
Fold 10 Valid RMSE: 2.132075391907796
Fold 10 Valid MSE:  4.545745476778782
Fold 10 Valid MAE:  1.6025330645749847


100%|██████████| 10/10 [10:58<00:00, 65.83s/it]


RMSE 평균 : 2.1306 



Valid MAE,█▄▁▆▇▃▃▇▅▆
Valid MSE,▅▁▃▄▆▆▅█▅▅
Valid RMSE,▅▁▃▄▆▆▅█▅▅
Valid MAE,1.60253
Valid MSE,4.54575
Valid RMSE,2.13208
param,"{""learning_rate"": 0...."


In [36]:
sub

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [37]:
sub['rating'] = (pred_df['pred_0'] + pred_df['pred_1'] + pred_df['pred_2'] + pred_df['pred_3'] + pred_df['pred_4'] + 
                               pred_df['pred_5'] + pred_df['pred_6'] + pred_df['pred_7'] + pred_df['pred_8'] + pred_df['pred_9']) / 10
submit = sub[['user_id', 'isbn', 'rating']]
submit

,user_id,isbn,rating
0,11676,0002005018,6.611996
1,116866,0002005018,7.031770
2,152827,0060973129,7.409333
3,157969,0374157065,7.469658
4,67958,0399135782,7.640977
...,...,...,...
76694,278543,1576734218,6.303957
76695,278563,3492223710,5.997220
76696,278633,1896095186,6.233525
76697,278668,8408044079,5.225945


In [38]:
submit.to_csv('submit.csv', index = False)